In [1]:
!pip install scikit-learn

In [13]:
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import models, layers
import torch
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
df = pd.read_csv('heart_disease.csv')
df

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,52,1,0,125,212,0,1,168,0,1.0,2,2,3,0
1,53,1,0,140,203,1,0,155,1,3.1,0,0,3,0
2,70,1,0,145,174,0,1,125,1,2.6,0,0,3,0
3,61,1,0,148,203,0,1,161,0,0.0,2,1,3,0
4,62,0,0,138,294,1,1,106,0,1.9,1,3,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1020,59,1,1,140,221,0,1,164,1,0.0,2,0,2,1
1021,60,1,0,125,258,0,0,141,1,2.8,1,1,3,0
1022,47,1,0,110,275,0,0,118,1,1.0,1,1,2,0
1023,50,0,0,110,254,0,0,159,0,0.0,2,0,2,1


In [4]:
number_columns = df.select_dtypes(include='number').columns.tolist()
number_columns

['age',
 'sex',
 'cp',
 'trestbps',
 'chol',
 'fbs',
 'restecg',
 'thalach',
 'exang',
 'oldpeak',
 'slope',
 'ca',
 'thal',
 'target']

In [5]:
len(number_columns)

14

In [6]:
for col in number_columns:
    if col == "fbs":
        continue
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)    
    IQR = Q3 -Q1
    lower_bound = Q1 - 1.5*IQR
    upper_bound = Q3 + 1.5*IQR
    df[col] = df[col].clip(lower=lower_bound, upper=upper_bound)

In [8]:
X = df.loc[:,"age":"thal"]
Y = df[['target']]
Y

,target
0,0
1,0
2,0
3,0
4,0
...,...
1020,1
1021,0
1022,0
1023,1


In [10]:
X_mean = X.mean()
X_std = X.std()
X_std

age          9.072290
sex          0.460373
cp           1.029641
trestbps    16.532208
chol        47.746162
fbs          0.356527
restecg      0.527878
thalach     22.881210
exang        0.472772
oldpeak      1.141865
slope        0.617755
ca           0.890414
thal         0.609123
dtype: float64

In [29]:
Y_test

,target
527,1
359,1
447,0
31,1
621,0
...,...
832,1
796,1
644,1
404,0


In [14]:
X_norm=(X-X_mean)/X_std
X_train, X_test,Y_train,Y_test=train_test_split(X_norm,Y,test_size=0.2, random_state=42)
X_train_tensor=torch.tensor(X_train.values, dtype=torch.float32)
X_train_tensor

tensor([[-0.5990,  0.6612,  1.0271,  ...,  0.9949,  2.0281, -0.5374],
        [ 1.0544, -1.5110, -0.9153,  ...,  0.9949, -0.7796, -0.5374],
        [-0.0479, -1.5110,  1.0271,  ...,  0.9949, -0.7796, -0.5374],
        ...,
        [-0.3785,  0.6612, -0.9153,  ...,  0.9949, -0.7796,  1.1043],
        [-1.2603,  0.6612, -0.9153,  ...,  0.9949, -0.7796,  1.1043],
        [-0.2683,  0.6612, -0.9153,  ...,  0.9949,  0.3435, -0.5374]])

In [32]:
model=models.Sequential([
    layers.Dense(30, input_dim = X.shape[1],
    kernel_initializer = tf.keras.initializers.HeNormal()),
    layers.BatchNormalization(),layers.ReLU(),
    layers.Dropout(0.5),layers.Dense(30,kernel_initializer= tf.keras.initializers.HeNormal()),
    layers.BatchNormalization(),layers.ReLU(),
    layers.Dropout(0.5),layers.Dense(30,kernel_initializer= tf.keras.initializers.HeNormal()),
    layers.BatchNormalization(),layers.ReLU(),
    layers.Dropout(0.5),layers.Dense(1,activation='sigmoid',kernel_initializer= tf.keras.initializers.GlorotNormal()),
])

In [35]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

In [36]:
model.fit(
    X_train,    # 학습에 사용할 입력 데이터
    Y_train,    # 학습에 사용할 정답 데이터

    epochs=1000,   # 전체 데이터를 1000번 반복 학습
   
    batch_size=64, # 한 번에 64개씩 데이터를 가져와 학습
  

    verbose=1,     # 학습 과정을 얼마나 자세히 출력할지 (1: 상세 출력)

    validation_data=(X_test, Y_test)
   
)

Epoch 1/1000
13/13 [==============================] - 1s 13ms/step - loss: 0.6978 - accuracy: 0.6073 - val_loss: 0.4994 - val_accuracy: 0.7561
Epoch 2/1000
13/13 [==============================] - 0s 3ms/step - loss: 0.5615 - accuracy: 0.7073 - val_loss: 0.4678 - val_accuracy: 0.8049
Epoch 3/1000
13/13 [==============================] - 0s 3ms/step - loss: 0.4866 - accuracy: 0.7793 - val_loss: 0.4866 - val_accuracy: 0.8293
Epoch 4/1000
13/13 [==============================] - 0s 2ms/step - loss: 0.4421 - accuracy: 0.8085 - val_loss: 0.5104 - val_accuracy: 0.8244
Epoch 5/1000
13/13 [==============================] - 0s 2ms/step - loss: 0.4460 - accuracy: 0.8037 - val_loss: 0.4958 - val_accuracy: 0.8244
Epoch 6/1000
13/13 [==============================] - 0s 3ms/step - loss: 0.4085 - accuracy: 0.8256 - val_loss: 0.5024 - val_accuracy: 0.8390
Epoch 7/1000
13/13 [==============================] - 0s 3ms/step - loss: 0.3802 - accuracy: 0.8415 - val_loss: 0.4966 - val_accuracy: 0.8341
Epoch

In [38]:
loss, acc = model.evaluate(X_test,Y_test)
print(f"\n 테스트 정확도: {acc * 100:.4f}%")
print(f"\n test :{loss:.4f}")

7/7 [==============================] - 0s 2ms/step - loss: 0.1183 - accuracy: 0.9073

 테스트 정확도: 90.7317%

 test :0.1183


In [39]:
new_patient_1 = pd.DataFrame({
    'age': [59],        # 나이
    'sex': [0],         # 성별 (0: 여성)
    'cp': [0],          # 가슴통증 유형
    'trestbps': [98],   # 안정 시 혈압
    'chol': [253],      # 콜레스테롤
    'fbs': [0],         # 공복 혈당
    'restecg': [0],     # 안정 시 심전도
    'thalach': [119],   # 최대 심박수
    'exang': [0],       # 운동 중 협심증
    'oldpeak': [1.0],   # ST절 하강
    'slope': [1],       # ST절 기울기
    'ca': [0],          # 주요 혈관 수
    'thal': [2]         # 결함 유형
})

# 데이터 출력
print(new_patient_1)

   age  sex  cp  trestbps  chol  fbs  restecg  thalach  exang  oldpeak  slope  \
0   59    0   0        98   253    0        0      119      0      1.0      1   

   ca  thal  
0   0     2  


In [41]:
new_patient_1_scaled= (new_patient_1- X_mean)/ X_std
new_patient_1_scaled

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal
0,0.503275,-1.510969,-0.915309,-2.01186,0.167941,-0.418674,-1.003559,-1.317814,-0.71194,-0.054511,-0.623816,-0.779577,-0.537358


In [42]:
pred_prob = model.predict(new_patient_1_scaled)
print(f"\n 환자 1의 심장병 예측 확률: {pred_prob}")

1/1 [==============================] - 0s 84ms/step

 환자 1의 심장병 예측 확률: [[0.9865069]]
